Add in necessary libraries

In [ ]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

Import Nasa data my requesting data through API. I wanted to get the parameters I thought were important which was precipitation, temperature, windspeed, and frost.

In [71]:
import requests
import pandas as pd

def get_nasa_power_data(lat, lon, start_date, end_date):
    """
    Fetches NASA POWER API data for given latitude, longitude, and time range.

    Args:
    - lat (float): Latitude of the location.
    - lon (float): Longitude of the location.
    - start_date (str): Start date in YYYYMMDD format.
    - end_date (str): End date in YYYYMMDD format.

    Returns:
    - Pandas DataFrame with selected weather parameters.
    """

    # Specify multiple parameters in the API request
    parameters = "PRECSNO,T2MDEW,PRECTOTCORR,T2M,WS2M"

    url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        "parameters": parameters,
        "community": "RE",
        "longitude": lon,
        "latitude": lat,
        "start": start_date,
        "end": end_date,
        "format": "JSON"
    }

    response = requests.get(url, params=params)

    data = response.json()

    # Convert JSON response to DataFrame and transpose it
    nasa_weather = pd.DataFrame.from_dict(data["properties"]["parameter"], orient="index").T

    # Reset index and rename date column
    nasa_weather.reset_index(inplace=True)
    nasa_weather.rename(columns={"index": "date"}, inplace=True)

    # Convert date column to proper datetime format
    nasa_weather["date"] = pd.to_datetime(nasa_weather["date"], format="%Y%m%d", errors="coerce")
    nasa_weather.dropna(subset=["date"], inplace=True)  # Remove invalid date rows

    nasa_weather.rename(columns={
        "PRECSNO": "Snow_Precipitation",
        "T2MDEW": "Dew_Point_2m",
        "PRECTOTCORR": "Total_Precipitation_mm",
        "T2M": "Temperature_2m_C",
        "WS2M": "Wind_Speed_2m"
    }, inplace=True)
    
    # Add Rounded_Lat and Rounded_Lng for merging
    nasa_weather['Rounded_Lat'] = lat
    nasa_weather['Rounded_Lng'] = lon
    
    # Display DataFrame
    print(f"\n Weather Data for Latitude {lat}, Longitude {lon}\n")
    display(nasa_weather)  # Works in Jupyter Notebook

    return nasa_weather

# Example: Fetch data for different locations
df_la = get_nasa_power_data(34.05, -118.25, "20240101", "20240107")  # Los Angeles
df_nyc = get_nasa_power_data(40.71, -74.01, "20240101", "20240107")  # New York City
df_sf = get_nasa_power_data(37.77, -122.42, "20240101", "20240107")  # San Francisco



 Weather Data for Latitude 34.05, Longitude -118.25



,date,Snow_Precipitation,Dew_Point_2m,Total_Precipitation_mm,Temperature_2m_C,Wind_Speed_2m,Rounded_Lat,Rounded_Lng
0,2024-01-01,0.0,6.12,0.04,12.14,1.64,34.05,-118.25
1,2024-01-02,0.0,7.05,0.09,11.55,1.59,34.05,-118.25
2,2024-01-03,0.0,6.75,5.87,10.86,3.25,34.05,-118.25
3,2024-01-04,0.0,2.14,0.02,9.76,2.89,34.05,-118.25
4,2024-01-05,0.0,1.99,0.00,10.80,2.00,34.05,-118.25
5,2024-01-06,0.0,1.23,0.12,10.50,2.86,34.05,-118.25
6,2024-01-07,0.0,-0.09,0.26,7.77,5.29,34.05,-118.25



 Weather Data for Latitude 40.71, Longitude -74.01



,date,Snow_Precipitation,Dew_Point_2m,Total_Precipitation_mm,Temperature_2m_C,Wind_Speed_2m,Rounded_Lat,Rounded_Lng
0,2024-01-01,0.00,2.07,0.04,4.98,2.65,40.71,-74.01
1,2024-01-02,0.00,-1.32,0.00,2.20,4.09,40.71,-74.01
2,2024-01-03,0.00,-0.04,0.00,3.46,3.09,40.71,-74.01
3,2024-01-04,0.00,-0.60,0.01,3.29,4.93,40.71,-74.01
4,2024-01-05,0.00,-3.65,0.00,0.51,4.52,40.71,-74.01
5,2024-01-06,0.86,0.18,30.07,3.13,5.14,40.71,-74.01
6,2024-01-07,0.09,2.15,7.85,3.27,6.36,40.71,-74.01



 Weather Data for Latitude 37.77, Longitude -122.42



,date,Snow_Precipitation,Dew_Point_2m,Total_Precipitation_mm,Temperature_2m_C,Wind_Speed_2m,Rounded_Lat,Rounded_Lng
0,2024-01-01,0.0,8.43,0.09,10.87,1.29,37.77,-122.42
1,2024-01-02,0.0,9.90,11.01,11.44,2.69,37.77,-122.42
2,2024-01-03,0.0,8.37,2.65,10.38,2.86,37.77,-122.42
3,2024-01-04,0.0,8.80,0.10,10.35,2.11,37.77,-122.42
4,2024-01-05,0.0,8.66,0.08,11.10,1.73,37.77,-122.42
5,2024-01-06,0.0,7.94,4.30,9.32,3.29,37.77,-122.42
6,2024-01-07,0.0,4.05,0.08,7.51,2.79,37.77,-122.42


Make sure that you downloaded the accident data and then import it sas a dataframe

In [5]:
us_accidents = pd.read_csv('US_Accidents_March23.csv')
us_accidents.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


Clean the accident data, I just took the location and severity because the NASA data already has the weather?

In [46]:
import pandas as pd
import numpy as np

def prepare_data(df):
    """
    Prepares the US Accidents DataFrame for merging with NASA weather data, keeping only necessary columns.
    
    Args:
    - df (DataFrame): Raw US Accidents dataset.

    Returns:
    - DataFrame: Processed DataFrame with 'date', 'Rounded_Lat', 'Rounded_Lng', and 'Severity' columns.
    """
    # Drop duplicates
    df.drop_duplicates(inplace=True)
    
    # Convert time columns to datetime format
    df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')

    # Remove rows with invalid 'Start_Time' values
    df = df[df['Start_Time'].notnull()].copy()

    # Extract 'date' from 'Start_Time' for merging with NASA weather data
    df['date'] = df['Start_Time'].dt.date

    # Filter for coordinates within LA County
    df = df[(df['Start_Lat'].between(33.7, 34.8)) & (df['Start_Lng'].between(-119.0, -117.6))]

    # Round latitude and longitude to 2 decimal places for approximate matching
    df['Rounded_Lat'] = df['Start_Lat'].round(2)
    df['Rounded_Lng'] = df['Start_Lng'].round(2)

    # Keep only relevant columns for merging
    df = df[['date', 'Rounded_Lat', 'Rounded_Lng', 'Severity']]

    return df


In [47]:
new_us_accidents = prepare_data(us_accidents)
new_us_accidents

,date,Rounded_Lat,Rounded_Lng,Severity
42863,2016-06-21,34.27,-118.64,2
42864,2016-06-21,34.32,-118.50,3
42865,2016-06-21,34.15,-118.37,2
42866,2016-06-21,34.08,-118.29,2
42867,2016-06-21,34.09,-118.24,3
...,...,...,...,...
7728384,2019-08-23,34.06,-118.00,2
7728387,2019-08-23,33.74,-117.84,2
7728388,2019-08-23,34.24,-118.42,2
7728391,2019-08-23,33.78,-117.85,2


I just put an example for UCLA and this week , this is what we will get from user input?

In [78]:
# Fetch NASA weather data for UCLA (Jan 1, 2025 - Jan 7, 2025)
nasa_weather_ucla = get_nasa_power_data(34.0689, -118.4452, "20161004", "20161125")



 Weather Data for Latitude 34.0689, Longitude -118.4452



,date,Snow_Precipitation,Dew_Point_2m,Total_Precipitation_mm,Temperature_2m_C,Wind_Speed_2m,Rounded_Lat,Rounded_Lng
0,2016-10-04,0.0,9.26,0.00,19.46,2.11,34.0689,-118.4452
1,2016-10-05,0.0,9.71,0.00,19.11,1.84,34.0689,-118.4452
2,2016-10-06,0.0,4.99,0.00,20.90,2.24,34.0689,-118.4452
3,2016-10-07,0.0,5.16,0.00,23.79,2.02,34.0689,-118.4452
4,2016-10-08,0.0,6.01,0.00,23.80,1.56,34.0689,-118.4452
5,2016-10-09,0.0,8.22,0.00,23.40,1.77,34.0689,-118.4452
6,2016-10-10,0.0,10.49,0.00,21.79,2.44,34.0689,-118.4452
7,2016-10-11,0.0,12.94,0.00,16.79,2.29,34.0689,-118.4452
8,2016-10-12,0.0,12.65,0.00,16.47,2.06,34.0689,-118.4452
9,2016-10-13,0.0,12.91,0.10,17.30,1.48,34.0689,-118.4452


In [79]:
import sqlite3

# Create a persistent SQLite database
conn = sqlite3.connect("accidents_weather_ucla.db")

# Ensure 'date' columns are strings for SQL
new_us_accidents['date'] = new_us_accidents['date'].astype(str)
nasa_weather_ucla['date'] = nasa_weather_ucla['date'].astype(str)

# Load datasets into the SQLite database
new_us_accidents.to_sql("accident_data", conn, index=False, if_exists="replace")
nasa_weather_ucla.to_sql("weather_data", conn, index=False, if_exists="replace")

# SQL query to merge on 'date', 'Rounded_Lat', and 'Rounded_Lng'
query = """
SELECT a.date, a.Rounded_Lat, a.Rounded_Lng, a.Severity,
       w.Total_Precipitation_mm AS Total_Precipitation_mm,
       w.Temperature_2m_C AS Temperature_2m_C,
       w.Wind_Speed_2m AS Wind_Speed_2m,
       w.Snow_Precipitation AS Snow_Precipitation,
       w.Dew_Point_2m AS Dew_Point_2m
FROM accident_data a
LEFT JOIN weather_data w
ON a.date = w.date AND a.Rounded_Lat = w.Rounded_Lat AND a.Rounded_Lng = w.Rounded_Lng
"""

# Execute the query and fetch the merged result
merged_data = pd.read_sql_query(query, conn)

# Display the merged data
print(merged_data.head())

# Close the database connection
conn.close()



         date  Rounded_Lat  Rounded_Lng  Severity Total_Precipitation_mm  \
0  2016-06-21        34.27      -118.64         2                   None   
1  2016-06-21        34.32      -118.50         3                   None   
2  2016-06-21        34.15      -118.37         2                   None   
3  2016-06-21        34.08      -118.29         2                   None   
4  2016-06-21        34.09      -118.24         3                   None   

  Temperature_2m_C Wind_Speed_2m Snow_Precipitation Dew_Point_2m  
0             None          None               None         None  
1             None          None               None         None  
2             None          None               None         None  
3             None          None               None         None  
4             None          None               None         None  


idk whats wrong with the above and why its calling for 2016? I made sure the date range i put exists in the us_accidents data as shown below.

In [80]:
print(new_us_accidents['date'].min(), new_us_accidents['date'].max())


2016-03-22 2023-03-31
